In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import pandas as pd
import medmnist
from medmnist import INFO, Evaluator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from scipy import stats
from PIL import Image
import pdb
import math

/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [34]:
# Example usage:
image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)
image


tensor([[[[ 9., 11.,  8., 20.],
          [18., 18., 17.,  3.],
          [10.,  7., 18., 16.],
          [18., 17., 22., 13.]]]])

In [49]:
r = 3
window_size = 3

# Extract 2x2 horizontal windows
windows_horizontal = []
Lr_all_horizontal = []

for i in range(image.size(2) - window_size + 1):
    for j in range(image.size(3) - window_size + 1):
        window = image[:, :, i:i+window_size, j:j+window_size]
        windows_horizontal.append(window)

        max_pool = nn.MaxPool2d(kernel_size=window.shape[2])
        max_pool_output = max_pool(window)
        min_pool_output = -max_pool(-window)
        nr = math.ceil(max_pool_output.item() / r) - math.ceil(min_pool_output.item() / r) - 1
        print(nr)
        Mr = torch.sum(torch.tensor(nr))
        Q_mr = nr / (window_size - r + 1)
        print(Mr)
        L_r = (Mr.item()**2) * Q_mr / (Mr * Q_mr)**2
        Lr_all_horizontal.append(L_r)

# Extract 2x2 vertical windows
windows_vertical = []
Lr_all_vertical = []

for j in range(image.size(3) - window_size + 1):
    for i in range(image.size(2) - window_size + 1):
        window = image[:, :, i:i+window_size, j:j+window_size]
        windows_vertical.append(window)

        max_pool = nn.MaxPool2d(kernel_size=window.shape[2])
        max_pool_output = max_pool(window)
        min_pool_output = -max_pool(-window)
        nr = math.ceil(max_pool_output.item() / r) - math.ceil(min_pool_output.item() / r) - 1
        Mr = torch.sum(torch.tensor(nr))
        Q_mr = nr / (window_size - r + 1)
        L_r = (Mr.item()**2) * Q_mr / (Mr * Q_mr)**2
        Lr_all_vertical.append(L_r)

print(Lr_all_horizontal)
print(Lr_all_vertical)

2
tensor(2)
5
tensor(5)
4
tensor(4)
6
tensor(6)
[tensor(0.5000), tensor(0.2000), tensor(0.2500), tensor(0.1667)]
[tensor(0.5000), tensor(0.2500), tensor(0.2000), tensor(0.1667)]


In [46]:
# Combine results to create a 9x9 matrix
Lr_horizontal_array = np.stack([tensor.numpy() for tensor in Lr_all_horizontal])
Lr_vertical_array = np.stack([tensor.numpy() for tensor in Lr_all_vertical])
combined_matrix = np.outer(Lr_horizontal_array, Lr_vertical_array)
combined_matrix = torch.tensor(combined_matrix)
combined_matrix = combined_matrix.unsqueeze(0).unsqueeze(0)
print(combined_matrix)

tensor([[[[0.2500, 0.1250, 0.1000, 0.0833],
          [0.1000, 0.0500, 0.0400, 0.0333],
          [0.1250, 0.0625, 0.0500, 0.0417],
          [0.0833, 0.0417, 0.0333, 0.0278]]]])


In [7]:
import torch
import torch.nn as nn
import math
import numpy as np

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        windows_horizontal = []
        Lr_all_horizontal = []

        window = image.unfold(1,3,1)
        print(window)
        windows_horizontal.append(window)

        max_pool = nn.MaxPool2d(kernel_size=window.shape[2])
        max_pool_output = max_pool(window)
        min_pool_output = -max_pool(-window)
        nr = math.ceil(max_pool_output / self.r) - math.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(torch.tensor(nr))
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr.item()**2) * Q_mr / (Mr * Q_mr)**2
        Lr_all_horizontal.append(L_r)

        windows_vertical = []
        Lr_all_vertical = []

        for j in range(image.size(3) - self.window_size + 1):
            for i in range(image.size(2) - self.window_size + 1):
                window = image[:, :, i:i+self.window_size, j:j+self.window_size]
                windows_vertical.append(window)

                max_pool = nn.MaxPool2d(kernel_size=window.shape[2])
                max_pool_output = max_pool(window)
                min_pool_output = -max_pool(-window)
                nr = math.ceil(max_pool_output / self.r) - math.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(torch.tensor(nr))
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr.item()**2) * Q_mr / (Mr * Q_mr)**2
                Lr_all_vertical.append(L_r)

        Lr_horizontal_array = np.stack([tensor.numpy() for tensor in Lr_all_horizontal])
        Lr_vertical_array = np.stack([tensor.numpy() for tensor in Lr_all_vertical])
        combined_matrix = np.outer(Lr_horizontal_array, Lr_vertical_array)
        combined_matrix = torch.tensor(combined_matrix)
        combined_matrix = combined_matrix.unsqueeze(0).unsqueeze(0)

        return combined_matrix

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)
image = image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)
# Assuming 'your_image_tensor' is your input tensor
result = custom_pooling_layer(image)
print(result)


RuntimeError: maximum size for tensor at dimension 1 is 1 but size is 2